In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from pathlib import Path


def load_data(class_type, tfms=None, size=224, bs = 16, path_splits='../splits.csv',
              path_jpg=Path('../../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/')):
    df = pd.read_csv(path_splits)

    if class_type == 'base':
        train_idx = df.index[df['split'] == 'base_train']
        valid_idx = df.index[df['split'] == 'base_validate']
    elif class_type == 'novel':
        train_idx = df.index[df['split'] == 'novel_train']
        valid_idx = df.index[df['split'] == 'novel_validate']
    else:
        raise Exception("Invalid class type input")

    ret_data = (ImageList.from_df(df, path_jpg)
                .split_by_idxs(train_idx, valid_idx)
                .label_from_df(cols='labels')
                .transform(tfms=tfms, size=size, resize_method=ResizeMethod.SQUISH)
                .databunch(bs=bs)
                .normalize(imagenet_stats))

    return ret_data


In [41]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import sys
sys.path.append('../..')
from scripts.data_scripts import *
from scripts.layers import *

# Parameters for the datablock API
tfms=None
size=224
bs = 16
path_splits='../splits.csv'
path_jpg=Path('../../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/')

# Load in novel class data
data = load_data('novel')

# Create Learner
arch = models.densenet161
learn = cnn_learner(data, arch, metrics=[error_rate, accuracy], loss_func=torch.nn.CrossEntropyLoss())

# Load model
model_path = '../../../../../home/ilu3/rl80/Monash_NIH_FYP/models/baseline/stage2_class_10to3'
learn = learn.load(model_path)

# Get samples per class to create weights
# Get index for the relevant labels
df = pd.read_csv(path_splits)
train_idx = []
train_idx.append(df.index[(df['split'] == 'novel_train') & (df['labels'] == 'Enlarged Cardiomediastinum')])
train_idx.append(df.index[(df['split'] == 'novel_train') & (df['labels'] == 'Lung Lesion')])
train_idx.append(df.index[(df['split'] == 'novel_train') & (df['labels'] == 'Pleural Effusion')])

# Hook to get the output of the second last layer
def feature_hook(model, input, output):
    global activation
    activation = torch.cat((activation, output), 0)
    
hook = learn.model[-1][-2].register_forward_hook(feature_hook)

data_novel = []
weight = torch.FloatTensor([]).cuda()
for i in range(3):
    # Create a list containing only samples from each label
    data_novel.append((ImageList.from_df(df, path_jpg)
                 .split_by_idxs(train_idx[i], train_idx[i])
                 .label_from_df(cols='labels')
                 .transform(tfms=tfms, size=size, resize_method=ResizeMethod.SQUISH)
                 .databunch(bs=bs)
                 .normalize(imagenet_stats))
               )
    
    # Get Activations
    activation = torch.FloatTensor([]).cuda()
    learn.validate(data_novel[i].valid_dl) # Use validation set as the training set is a multiple of the batch size
    mean = torch.mean(activation, 0)
    weight = torch.cat((weight, mean[None]), 0)

# Replace the layer parameters with the new feature average
param = nn.Parameter(weight, True)
learn.layer_groups[-1][-1].weight = param
learn.save('stage3')

FileNotFoundError: [Errno 2] No such file or directory: '../../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/models/models/stage3.pth'

torch.Size([3, 512])

In [20]:
list(learn.model.parameters())[-2]

Parameter containing:
tensor([[-0.0284, -0.0016,  0.0239,  ..., -0.0205,  0.0300,  0.0006],
        [ 0.0402, -0.0078,  0.0059,  ...,  0.0379,  0.0095,  0.0173],
        [ 0.0153,  0.0122, -0.0195,  ..., -0.0247,  0.0242,  0.0106]],
       device='cuda:0', requires_grad=True)

In [14]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import sys
sys.path.append('../..')
from scripts.data_scripts import *
from scripts.layers import *

# Load in base class data
data = load_data('novel')

# Use custom header to replace classifier layers
custom_head = create_head(4416, 3, lin_ftrs=[512])
custom_head[-1] = CosineSimilarityLayer(512, 3)

# Create Learner
arch = models.densenet161
learn = cnn_learner(data, arch, metrics=[accuracy],
                    loss_func=torch.nn.CrossEntropyLoss(),
                    custom_head=custom_head)

# Load model
model_path = 'stage3_test'
learn = learn.load(model_path, strict=False) # Need strict=False to ignore the extra bias parameters

# Results
results = learn.validate(data.valid_dl)
with open('test.txt', 'a') as f:
    print('Accuracy:', round(results[1].item(),4), file=f)

In [10]:
with open('test.txt', 'a') as f:
    print('Accuracy:', file=f)
    print('Accuracy:', file=f)
    print('Accuracy:', file=f)

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import sys
sys.path.append('../..')
from scripts.data_scripts import *
from scripts.layers import *

# Parameters for the datablock API
tfms=None
size=224
bs = 16
path_splits='../splits.csv'
path_jpg=Path('../../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/')

# Load in novel class data
data = load_data('novel')
data_base = load_data('base')

# Get samples per class to create weights
# Get index for the relevant labels
df = pd.read_csv(path_splits)
train_idx = []
train_idx.append(df.index[(df['split'] == 'novel_train') & (df['labels'] == 'Enlarged Cardiomediastinum')])
train_idx.append(df.index[(df['split'] == 'novel_train') & (df['labels'] == 'Lung Lesion')])
train_idx.append(df.index[(df['split'] == 'novel_train') & (df['labels'] == 'Pleural Effusion')])

# Hook to get the output of the second last layer
def feature_hook(model, input, output):
    global activation
    activation = torch.cat((activation, output), 0)

for k in range(30):
    # Create Learner
    num_sample = [2592, 2850, 388, 1926, 328, 4327, 4000, 762, 823, 1860]
    max_sample = max(num_sample)
    weight = torch.FloatTensor([max_sample / x for x in num_sample]).cuda()

    arch = models.densenet161
    learn = cnn_learner(data_base, arch, metrics=[error_rate, accuracy],
                        loss_func=torch.nn.CrossEntropyLoss(weight=weight))

    # Load Model and convert last layer from 10 classes to 3
    model_path = f"bestmodel_{k}"
    learn = learn.load(model_path)
    learn.model[-1][-1] = nn.Linear(512, 3)
    learn.save('stage2')
    
    # Use custom header to replace classifier layers
    custom_head = create_head(4416, 3, lin_ftrs=[512])
    custom_head[-1] = CosineSimilarityLayer(512, 3)

    # Create Learner
    arch = models.densenet161
    learn = cnn_learner(data, arch, metrics=[accuracy],
                        loss_func=torch.nn.CrossEntropyLoss(),
                        custom_head=custom_head)
    
    # Load model
    learn = learn.load('stage2', strict=False)
    hook = learn.model[-1][-2].register_forward_hook(feature_hook)

    data_novel = []
    weight = torch.FloatTensor([]).cuda()
    for i in range(3):
        # Create a list containing only samples from each label
        data_novel.append((ImageList.from_df(df, path_jpg)
                     .split_by_idxs(train_idx[i], train_idx[i])
                     .label_from_df(cols='labels')
                     .transform(tfms=tfms, size=size, resize_method=ResizeMethod.SQUISH)
                     .databunch(bs=bs)
                     .normalize(imagenet_stats))
                   )

        # Get Activations
        activation = torch.FloatTensor([]).cuda()
        learn.validate(data_novel[i].valid_dl) # Use validation set as the training set is a multiple of the batch size
        mean = torch.mean(activation, 0)
        weight = torch.cat((weight, mean[None]), 0)
        
    hook.remove()
    # Replace the layer parameters with the new feature average
    param = nn.Parameter(weight, True)
    learn.layer_groups[-1][-1].weight = param
    # Results
    results = learn.validate(data.valid_dl)
    with open('test.txt', 'a') as f:
        print(k, ': Accuracy:', round(results[1].item(),4), file=f)

In [3]:
weight

tensor([[-0.1333,  0.1624, -0.1139,  ...,  0.0982, -0.0786, -0.0167],
        [-0.1461,  0.2213, -0.1593,  ...,  0.0467, -0.1567, -0.0174],
        [-0.1403,  0.1544, -0.1227,  ...,  0.0706, -0.0923,  0.0219]],
       device='cuda:0')

In [25]:
hook

In [34]:
learn.model[-1][-2].hooks

AttributeError: 'Dropout' object has no attribute 'hooks'